# Detección de sombras 

*: El "ejemplo nubes" sale de acá: https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

### Para ver las bandas del FC2_with_cloud_mask:

Alturas de nubes con S5: https://code.earthengine.google.com/448f9425387963431ec1ac567b206bc5; (https://code.earthengine.google.com/?scriptPath=users%2Fjumanbar%2FpruebasRN%3AAlturaNubes)

## Autenticación

In [1]:
import os
import ee
from google.auth.transport.requests import AuthorizedSession


##################################################################
# CREDENTIALS =======
print('\nEnviando credenciales a la nube ...')

PROJECT = 'pruebas-gee-00' # Ej: pruebas-engine-00
# SERVICE_ACCOUNT = 'pruebas-gee-jmb@pruebas-gee-00.iam.gserviceaccount.com'
SERVICE_ACCOUNT = 'jumanbar@gmail.com'
KEY = os.path.join(os.getcwd(), 'debian-key.json')
rest_api_url = 'https://earthengine.googleapis.com/v1beta/projects/{}/table:computeFeatures'

print('\LLAVE:', KEY)

# PROBLEMAS ACA / INICIO
credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])

session = AuthorizedSession(scoped_credentials)
ee.Initialize(credentials)
# PROBLEMAS ACA / FIN
###################################################################


Enviando credenciales a la nube ...
\LLAVE: c:\Users\juan.barreneche\Documents\gee_automatizacion\debian-key.json


## Variables y tal

In [2]:
"""
id_zona = 1
zona = 'PALMAR'
geom = ee.FeatureCollection('users/brunogda/zonas_palmar_represa_dis')\
    .first()\
    .geometry()

mask_ndwi = ee.Image(0)
lo_ndwi = ee.Image(0)
"""

from math import tan, pi, log10, ceil
from pprint import pprint
import numpy as np
from inifun import *
import calfun as cf
id_zona = 5
nzonas = 60
isValidZoneID(id_zona, nzonas)

def getAssetString(nzonas, id_zona):    
    if nzonas == 7:
        out = getAssetFromIdZona(id_zona)
    else:
        out = getAssetFromIdZona60(id_zona)
    return out

asset_string = getAssetString(nzonas, id_zona)

print(f'\tAsset string: {asset_string}')

# Polígono:
geom = ee.FeatureCollection(asset_string).first().geometry()

p = [10, 50, 90]
cloud_perc  = 50 # 25
cloud_perc2 = 50 # 25
# MAX_CLOUD_PROBABILITY = 10
MAX_CLOUD_PROBABILITY = 40
percentil_stat_base_nubes = 70;
escala_proy_nubes = 50

ini_date = '2023-05-17'
ini_date_o3 = '2023-05-17'
end_date = '2023-06-18'

# Cloud shadows
BUFFER = 50
cf.escala_proy_nubes = 50
cf.cbh_percentil = percentil_stat_base_nubes

# COPIAR VARIABLES AL NAMESPACE DE calfun:
cf.zona = 'RN60Z'
cf.id_zona = id_zona
cf.geom = geom
cf.p = p

cf.cloud_perc = cloud_perc
cf.cloud_perc2 = cloud_perc2
cf.MAX_CLOUD_PROBABILITY = MAX_CLOUD_PROBABILITY
cf.escala_proy_nubes = escala_proy_nubes

cf.end_date = end_date
cf.ini_date = ini_date
cf.ini_date_o3 = ini_date_o3

fprop = cf.filterProperties(60, 5)
print(f'\tid_zona:  {id_zona}')
print(f'\tnzonas:   {nzonas}')
print(f'\tProperty: {fprop["prp"]}')
print(f'\tValue:    {fprop["val"]}')

	Asset string: users/brunogda/RN60/05_rn
	id_zona:  5
	nzonas:   60
	Property: MGRS_TILE
	Value:    21HVD


## Distancia de la nube a su sombra (demo)

In [3]:
from datetime import datetime as dt, timezone

MSI_source = 'COPERNICUS/S2_HARMONIZED'
MSI = ee.ImageCollection(MSI_source)
print(f'ini: {ini_date} - fin: {end_date}')
common_criteria = ee.Filter.And(ee.Filter.bounds(geom),
                                ee.Filter.date(ini_date, end_date))

x = MSI.filter(common_criteria).filter(ee.Filter.eq(fprop['prp'], fprop['val']))
a = x.first().date().getInfo()['value']
print(f'Fecha de primer imagen de la colección S2:\n\t{dt.fromtimestamp(a/1000)}')
a2 = x.first().get('GENERATION_TIME').getInfo()

nubes = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CLOUD')\
        .filterDate(x.first().date().advance(-4, 'hour'), x.first().date().advance(4, 'hour'))

clouds_height = cf.cloudsBaseHeightByDate(x.first().date())
print(f'Altura de nubes (percentil {cf.cbh_percentil}): {clouds_height.getInfo():.2f}m')

d = cf.shadowDistFromClouds(x.first())

# print(datetime.fromtimestamp(x.first().date().getInfo()))
print(f'Distancia en el plano de la sombra a la nube: {d.getInfo():.2f}m => {ceil(d.getInfo()/cf.escala_proy_nubes)} px')

ini: 2023-05-17 - fin: 2023-06-18
Fecha de primer imagen de la colección S2:
	2023-05-18 11:01:46.657000
Altura de nubes (percentil 70): 1232.30m
Distancia en el plano de la sombra a la nube: 1990.31m => 40 px


In [4]:
print(dt.fromtimestamp(1686997020000/1000))
print(dt.fromtimestamp(1655901773000/1000))
print(dt.fromtimestamp(1687010506681/1000))
print(dt.fromtimestamp(1687010506681/1000, timezone.utc))

2023-06-17 07:17:00
2022-06-22 09:42:53
2023-06-17 11:01:46.681000
2023-06-17 14:01:46.681000+00:00


### Run

In [5]:
# Sentinel multibanda
MSI_source = 'COPERNICUS/S2_HARMONIZED'    # Nuestro, original
MSI = ee.ImageCollection(MSI_source)
# Probabilidad de nubes:
S2_clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

print(f'ini: {ini_date} - fin: {end_date}')

# Filter input collections by desired data range and region.
common_criteria = ee.Filter.And(ee.Filter.bounds(geom),
                                ee.Filter.date(ini_date, end_date))

S2_clouds = S2_clouds.filter(common_criteria)

# Esto se fija en qué imágenes de la colección hay pocas nubes en total y se
# queda con esas:
S2_mask = (
    MSI
    .filter(common_criteria)
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloud_perc2)
)

# FILTER Sentinel 2 collection
FC2 = S2_mask.filter(ee.Filter.eq(fprop["prp"], fprop["val"])).map(cf.maskEdges)

cf.printDatesFromImageCollection(FC2, 'FC2')

## NDWI
## Sirve con S2_HARMONIZED y S2_SR_HARMONIZED
ndwi = S2_mask.median().normalizedDifference(['B3', 'B8'])
hi_ndwi = ndwi.select('nd').gte(0.2)
lo_ndwi = ndwi.select('nd').lt(0.2)
mask_ndwi = hi_ndwi.updateMask(hi_ndwi)

# FILTER Sentinel 2 collection
# FC2 = MSI.filterDate(ini_date, end_date)\
#     .filterBounds(geom)\
#     .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloud_perc)\
#     .filter(ee.Filter.eq(prp, val))

# Join S2 SR with cloud probability dataset to add cloud mask.
FC2_with_cloud_mask = ee.ImageCollection(
    ee.Join.saveFirst('cloud_mask').apply(
        primary = FC2,
        secondary = S2_clouds,
        condition = ee.Filter.equals(
            leftField = 'system:index',
            rightField = 'system:index')
        )
    )

############## DESCOMENTAR ESTO:
S2_cloud_masked = FC2_with_cloud_mask.map(cf.add_cld_shdw_mask)

ini: 2023-05-17 - fin: 2023-06-18
Lista de fechas en FC2:
- 2023-05-18 11:01:46.657000
- 2023-05-23 11:01:45.040000
- 2023-05-28 11:01:46.336000
- 2023-05-30 10:51:49.891000
- 2023-06-12 11:01:46.012000
- 2023-06-14 10:51:51.080000
- 2023-06-17 11:01:46.681000
{'bands': [{'crs': 'EPSG:32721',
            'crs_transform': [50, 0, 399960, 0, 50, 6400000],
            'data_type': {'max': 1,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [2196, 2196],
            'id': 'cloudmask',
            'origin': [0, -2196]}],
 'type': 'Image'}


### Proyección nubes

Esta parte es extraída de una de las funciones (ver [Nuevas](#scrollTo=Nuevas)). Lo puse acá porque quería ver cómo funcionaba eso de la proyección y la función `directionalDistanceTransform`

In [6]:
# ! C:/Users/juan.barreneche/AppData/Local/Programs/Python/Programs/Python/Python310/python.exe -m pip install geemap
fecha_muestra = '2023-06-12'
import geemap
m = geemap.Map()
satimage = FC2.filterDate(f'{fecha_muestra}T00:00:01', f'{fecha_muestra}T23:59:59').first()
test = S2_cloud_masked.filterDate(f'{fecha_muestra}T00:00:01', f'{fecha_muestra}T23:59:59').first()
img_shadow = ee.Image(cf.add_shadow_bands(test))
d = cf.shadowDistFromClouds(test).getInfo()
# test = S2_cloud_masked.first()
cf.printBandNames(test)
maxdist = ceil(d/cf.escala_proy_nubes)
m.add_layer(satimage, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'FC2', True, 1)
m.add_layer(img_shadow.select('cloud_transform'), {'min': 0, 'max': maxdist, 'palette': ['yellow', 'red']}, 'DISTANCE', True, 1)
m.add_layer(test.mask(), None, 'test Clouds mask', False, 1)
m.set_zoom(12)
m.set_center(lat=-33.09039099883797, lon=-57.306747436523445)
display(m)

ee.ee_date.Date

NameError: name 'foo' is not defined